In [132]:
from dataclasses import dataclass
import random
from collections import Counter
from typing import Callable, Any

In [133]:
# cards & decks setup
RANKS = ['2', '3', '4', '5', '6', '7', '8', '9', 'T', 'J', 'Q', 'K', 'A'] # T = 10
SUITS = ['Spades', 'Hearts', 'Diamonds', 'Clubs']

RANK_POINTS = {
    '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, 
    '9': 9, 'T': 10, 'J': 11, 'Q': 12, 'K': 13, 'A': 14,
}

HAND_ORDER = [
    'high_card',
    'pair',
    'two_pair',
    'three_of_a_kind',
    'straight',
    'flush',
    'full_house',
    'four_of_a_kind',
    'straight_flush',
]

HAND_BASE_VALUES = {
    'high_card': (10, 1),
    'pair': (20, 1),
    'two_pair': (30, 1),
    'three_of_a_kind': (40, 2),
    'straight': (45, 2),
    'flush': (50, 2),
    'full_house': (60, 3),
    'four_of_a_kind': (80, 4),
    'straight_flush': (100, 5),
}

@dataclass(frozen = True)
class Card:
    rank: str
    suit: str

    def __str__(self):
        return f'{self.rank} of {self.suit}'

def new_deck(shuffle = True, seed = None):
    deck = [Card(rank = r, suit = s) for r in RANKS for s in SUITS]
    if shuffle:
        rng = random.Random(seed)
        rng.shuffle(deck)
    return deck


def draw_cards(deck, n):
    if n > len(deck):
        raise ValueError('Not enough cards in deck')
    drawn = deck[-n:] # from the top
    del deck[-n:]
    return drawn

# evaluation setup
def rank_values(cards):
    return [RANK_POINTS[c.rank] for c in cards]


def is_flush_func(cards):
    suits = {c.suit for c in cards}
    return len(suits) == 1

def is_straight_func(values):
    sorted_ranks = sorted(set(values))
    if len(sorted_ranks) != 5:
        return False, max(values)

    if sorted_ranks[-1] - sorted_ranks[0] == 4:
        return True, sorted_ranks[-1]

    if sorted_ranks == [2, 3, 4, 5, 14]: # cases where straight is A-2-3-4-5
        return True, 5

    return False, max(values)


def evaluate_hand(cards):
    if len(cards) != 5:
        raise ValueError('evaluate_hand expects exactly 5 cards only')

    values = rank_values(cards)
    value_counts = Counter(values)
    counts = sorted(value_counts.items(), key = lambda kv: (-kv[1], -kv[0]))

    is_flush = is_flush_func(cards)
    is_straight, straight_high = is_straight_func(values)

    # straight flush
    if is_flush and is_straight:
        hand_name = 'straight_flush'
        primary = [straight_high]
        kickers: list[int] = []
    else:
        # four of a kind
        pattern = sorted(value_counts.values(), reverse = True)
        # print(pattern)
        if pattern == [4, 1]:
            hand_name = 'four_of_a_kind'
            four_rank = counts[0][0]
            kicker_rank = [r for r, c in counts if r != four_rank][0]
            primary = [four_rank]
            kickers = [kicker_rank]
        # full house
        elif pattern == [3, 2]:
            hand_name = 'full_house'
            trip_rank = counts[0][0]
            pair_rank = counts[1][0]
            primary = [trip_rank, pair_rank]
            kickers = []
        # flush
        elif is_flush:
            hand_name = 'flush'
            primary = sorted(values, reverse = True)
            kickers = []
        # straight
        elif is_straight:
            hand_name = 'straight'
            primary = [straight_high]
            kickers = []
        # three of a kind
        elif pattern == [3, 1, 1]:
            hand_name = 'three_of_a_kind'
            trip_rank = counts[0][0]
            primary = [trip_rank]
            kickers = [r for r, _ in counts if r != trip_rank]
        # two pair
        elif pattern == [2, 2, 1]:
            hand_name = 'two_pair'
            pair1, pair2 = counts[0][0], counts[1][0]
            kicker = [r for r, c in counts if c == 1][0]
            primary = sorted([pair1, pair2], reverse = True)
            kickers = [kicker]
        # pair
        elif pattern == [2, 1, 1, 1]:
            hand_name = 'pair'
            pair_rank = counts[0][0]
            primary = [pair_rank]
            kickers = [r for r, _ in counts if r != pair_rank]
        # high
        else:
            hand_name = 'high_card'
            primary = [max(values)]
            kickers = sorted([r for r in values if r != primary[0]], reverse = True)

    rank_index = HAND_ORDER.index(hand_name)

    return {
        'name': hand_name,
        'rank_index': rank_index,
        'primary_ranks': primary,
        'kickers': kickers,
        'is_flush': is_flush,
        'is_straight': is_straight,
    }

# scoring setup
def hand_base_chips(hand_type):
    chips, _ = HAND_BASE_VALUES[hand_type]
    return chips

def hand_base_mult(hand_type):
    _, mult = HAND_BASE_VALUES[hand_type]
    return mult

def score_hand(cards, hand_info):
    hand_type = hand_info['name']
    base_chips = hand_base_chips(hand_type)
    base_mult = hand_base_mult(hand_type)

    card_chip_sum = sum(RANK_POINTS[c.rank] for c in cards)
    chips = base_chips + card_chip_sum
    mult = base_mult
    total = chips * mult
    return chips, mult, total


In [134]:
@dataclass
class BlindConfig:
    ante: int
    name: str
    target_chips: int
    hands: int
    discards: int

In [135]:
BLIND_NAMES = ['Small', 'Big', 'Boss']

def make_blinds_for_ante(ante):
    # base chip requirements for ante 1
    base_targets = {
        'Small': 250,
        'Big': 400,
        'Boss': 650,
    }
    # exponential ante scale
    scale = 1.4 ** (ante - 1)

    blinds = []
    for name in BLIND_NAMES:
        target = int(base_targets[name] * scale)
        # hands and disscards also scale
        if name == 'Small':
            hands = 5
            discards = 3
        elif name == 'Big':
            hands = 5
            discards = 3
        else: 
            # boss
            hands = 6
            discards = 3

        blinds.append(
            BlindConfig(
                ante = ante,
                name = name,
                target_chips = target,
                hands = hands,
                discards = discards,
            )
        )
    return blinds

In [136]:
# testing
def test_draw_and_score(seed):
    deck = new_deck(shuffle = True, seed = seed)
    hand = draw_cards(deck, 5)
    info = evaluate_hand(hand)
    chips, mult, total = score_hand(hand, info)

    card_str = ', '.join(str(c) for c in hand)
    print('Hand:', card_str)
    print('Type:', info['name'])
    print('Chips:', chips, '| Mult:', mult, '| Score:', total)

In [137]:
# set up example Jokers
@dataclass
class Joker:
    name: str
    description: str
    apply_func: Callable[[list[Card], dict, int, int, dict], tuple[int, int]]

    def apply(self, cards, hand_info, chips, mult, context):
        return self.apply_func(cards, hand_info, chips, mult, context)

FACE_RANKS = {'J', 'Q', 'K'}
PAIRISH_HANDS = {'pair', 'two_pair', 'three_of_a_kind', 'full_house', 'four_of_a_kind'}

def joker_jolly():
    def apply(cards, hand_info, chips, mult, context):
        if hand_info['name'] in PAIRISH_HANDS:
            chips += 25
        return chips, mult

    return Joker(
        name = 'Jolly Joker',
        description = 'If hand has at least a pair, +25 chips.',
        apply_func = apply,
    )


def joker_zany():
    def apply(cards, hand_info, chips, mult, context):
        if hand_info['name'] in PAIRISH_HANDS:
            mult += 3
        return chips, mult

    return Joker(
        name = 'Zany Joker',
        description = 'If hand has at least a pair, +3 mult.',
        apply_func = apply,
    )

def joker_green():
    def apply(cards, hand_info, chips, mult, context):
        run = context.get('run', {})
        hands_played = run.get('hands_played', 0)
        # to avoid being too insane, maybe scale slower
        bonus = hands_played
        mult += bonus
        return chips, mult

    return Joker(
        name = 'Green Joker',
        description = 'Gains +1 mult per hand played this run.',
        apply_func = apply,
    )

def joker_stuntman_lite():
    def apply(cards, hand_info, chips, mult, context):
        chips += 150
        return chips, mult

    return Joker(
        name = 'Stuntman Lite',
        description = '+150 chips.',
        apply_func = apply,
    )


def joker_supernova():
    def apply(cards, hand_info, chips, mult, context):
        run = context.get('run', {})
        counts = run.get('hand_type_counts', {})
        hand_type = hand_info['name']
        times_played = counts.get(hand_type, 0)
        mult += times_played
        return chips, mult

    return Joker(
        name = 'Supernova',
        description = 'Adds times this hand type has been played to Mult.',
        apply_func = apply,
    )

def joker_card_sharp():
    def apply(cards, hand_info, chips, mult, context):
        run = context.get('run', {})
        counts = run.get('hand_type_counts', {})
        hand_type = hand_info['name']
        if counts.get(hand_type, 0) > 1:
            mult *= 2
        return chips, mult

    return Joker(
        name = 'Card Sharp',
        description = 'x2 mult if this hand type has been played before.',
        apply_func = apply,
    )

def joker_smeared():
    def apply(cards, hand_info, chips, mult, context):
        if hand_info['is_flush']:
            mult *= 2
        return chips, mult

    return Joker(
        name = 'Smeared Joker',
        description = 'If hand is a flush, x2 mult.',
        apply_func = apply,
    )


def joker_arrowhead():
    def apply(cards, hand_info, chips, mult, context):
        spades = sum(1 for c in cards if c.suit == 'Spades')
        chips += 6 * spades
        return chips, mult

    return Joker(
        name = 'Arrowhead',
        description = '+6 chips per Spade in hand.',
        apply_func = apply,
    )


def joker_wrathful():
    def apply(cards, hand_info, chips, mult, context):
        hearts = sum(1 for c in cards if c.suit == 'Hearts')
        if hearts >= 3:
            mult += 4
        return chips, mult

    return Joker(
        name = 'Wrathful Joker',
        description = '+4 mult if hand has at least 3 Hearts.',
        apply_func = apply,
    )

def _is_almost_straight(values: list[int]) -> bool:
    # 'almost straight' if window of size 5 has range <= 5 and at least 4 distinct ranks
    vs = sorted(set(values))
    if len(vs) < 4:
        return False
    # check any window of up to 5 ranks
    for i in range(len(vs)):
        window = vs[i:i+5]
        if len(window) >= 4 and window[-1] - window[0] <= 5:
            return True
    return False


def joker_shortcut():
    def apply(cards, hand_info, chips, mult, context):
        values = [RANK_POINTS[c.rank] for c in cards]
        if hand_info['is_straight']:
            mult += 5
        elif _is_almost_straight(values):
            mult += 3
        return chips, mult

    return Joker(
        name = 'Shortcut',
        description = '+5 mult for a straight, +3 if almost-straight.',
        apply_func = apply,
    )


def joker_runner():
    def apply(cards, hand_info, chips, mult, context):
        run = context.get('run', {})
        counts = run.get('hand_type_counts', {})
        played_straights = counts.get('straight', 0) + counts.get('straight_flush', 0)
        chips += 10 * played_straights
        return chips, mult

    return Joker(
        name = 'Runner',
        description = '+10 chips per straight/straight_flush played this run.',
        apply_func = apply,
    )

def joker_pareidolia():
    def apply(cards, hand_info, chips, mult, context):
        run = context.get('run', {})
        run['pareidolia_active'] = True
        return chips, mult

    return Joker(
        name = 'Pareidolia',
        description = 'All cards are treated as faces for face-jokers.',
        apply_func = apply,
    )

def _count_face_like(cards, context):
    run = context.get('run', {})
    if run.get('pareidolia_active'):
        return len(cards)
    return sum(1 for c in cards if c.rank in FACE_RANKS)


def joker_scary_face():
    def apply(cards, hand_info, chips, mult, context):
        face_count = _count_face_like(cards, context)
        chips += 8 * face_count
        return chips, mult

    return Joker(
        name = 'Scary Face',
        description = '+8 chips per face card (or all cards with Pareidolia).',
        apply_func = apply,
    )

def joker_smiley_face():
    def apply(cards, hand_info, chips, mult, context):
        face_count = _count_face_like(cards, context)
        mult += face_count
        return chips, mult

    return Joker(
        name = 'Smiley Face',
        description = '+1 mult per face card (or all cards with Pareidolia).',
        apply_func = apply,
    )

def joker_bull():
    def apply(cards, hand_info, chips, mult, context):
        run = context.get('run', {})
        player_money = run.get('player_money', 0)
        chips += 2 * player_money
        return chips, mult

    return Joker(
        name = 'Bull',
        description = '+2 chips for each dollar you have.',
        apply_func = apply,
    )

def joker_bootstraps():
    def apply(cards, hand_info, chips, mult, context):
        run = context.get('run', {})
        player_money = run.get('player_money', 0)
        mult += (player_money // 5) * 2
        return chips, mult

    return Joker(
        name = 'Bootstraps',
        description = '+2 mult for every $5 you have.',
        apply_func = apply,
    )



In [138]:
ALL_JOKER_FACTORIES = [
    joker_supernova,
    joker_jolly,
    joker_zany,
    joker_green,
    joker_stuntman_lite,
    joker_card_sharp,
    joker_smeared,
    joker_arrowhead,
    joker_wrathful,
    joker_shortcut,
    joker_runner,
    joker_pareidolia,
    joker_scary_face,
    joker_smiley_face,
    joker_bull,
    joker_bootstraps,
]


In [139]:
@dataclass
class ShopItem:
    kind: str
    name: str
    price: int
    payload: Any
    description: str

In [140]:
HandChipsUpgrades = dict[str, int]
HandMultUpgrades = dict[str, int]

RunContext = dict[str, Any]
def new_run_context():
    return {
        'global_hand_index': 0,
        'hands_played': 0,
        'hand_type_counts': {hand_type: 0 for hand_type in HAND_ORDER},
        'pareidolia': False
    }



In [141]:
def apply_planet_upgrade(hand_type, upgrade_kind, chips_up, mult_up):
    if upgrade_kind == 'chips':
        chips_up[hand_type] = chips_up.get(hand_type, 0) + 10 # +10 chips per Planet
    elif upgrade_kind == 'mult':
        mult_up[hand_type] = mult_up.get(hand_type, 0) + 1 # +1 mult per Planet

In [142]:
def score_with_modifiers(cards, hand_info, jokers, hand_chips_upgrades, hand_mult_upgrades, context = None):
    if context is None:
        context = {}

    hand_type = hand_info['name']

    base_chips = hand_base_chips(hand_type)
    base_mult = hand_base_mult(hand_type)

    # apply Planet upgrades
    extra_chips = hand_chips_upgrades.get(hand_type, 0)
    extra_mult = hand_mult_upgrades.get(hand_type, 0)

    card_chip_sum = sum(RANK_POINTS[c.rank] for c in cards)
    chips = base_chips + extra_chips + card_chip_sum
    mult = base_mult + extra_mult

    # apply Jokers left to right
    for j in jokers:
        chips, mult = j.apply(cards, hand_info, chips, mult, context)

    total = chips * mult
    return chips, mult, total

In [143]:
# Joker scoring
def score_with_jokers(cards, hand_info, jokers, context = None):
    return score_with_modifiers(cards, hand_info, jokers,
                                hand_chips_upgrades={},
                                hand_mult_upgrades={},
                                context=context)

In [144]:
@dataclass
class PlayerState:
    hp: int = 3
    money: int = 0
    ante: int = 1
    blind_index: int = 0 # 0: Small, 1: Big, 2: Boss
    max_jokers: int = 3 # base Joker slots
    shop_discount: int = 0 # flat discount on shop prices

In [145]:
@dataclass
class RunResult:
    player: PlayerState
    jokers: list[Joker]
    hand_chips_upgrades: HandChipsUpgrades
    hand_mult_upgrades: HandMultUpgrades
    blinds_cleared: int
    max_ante_reached: int

In [146]:
def apply_interest(player):
    interest = min(5, player.money // 5)
    if interest > 0:
        player.money += interest

In [147]:
@dataclass
class Voucher:
    name: str
    description: str
    apply_func: Callable[[PlayerState], None]

    def apply(self, player):
        self.apply_func(player)

In [148]:
def voucher_extra_slot():
    def apply(player):
        player.max_jokers += 1
    return Voucher(
        name = 'Extra Slot',
        description = 'Gain +1 Joker slot.',
        apply_func = apply,
    )

def voucher_coupon():
    def apply(player):
        player.shop_discount += 1
    return Voucher(
        name = 'Coupon',
        description = 'All shop items cost 1$ less (min 1$).',
        apply_func = apply,
    )

ALL_VOUCHER_FACTORIES = [
    voucher_extra_slot,
    voucher_coupon,
]

In [149]:
@dataclass
class BossDebuff:
    name: str
    description: str
    apply_func: Callable[[list[Card], dict, int, int, dict], tuple[int, int]]

    def apply(self, cards, hand_info, chips, mult, context):
        return self.apply_func(cards, hand_info, chips, mult, context)

In [150]:
def boss_no_discards():
    def apply(cards, hand_info, chips, mult, context):
        # effect handled in play_blind by zeroing discards; scoring unchanged
        return chips, mult

    return BossDebuff(
        name='No Discards',
        description='You cannot discard any hands this blind.',
        apply_func=apply,
    )


def boss_debuff_hearts():
    def apply(cards, hand_info, chips, mult, context):
        # if you play any Hearts, total chips are penalized
        if any(c.suit == 'Hearts' for c in cards):
            chips = int(chips * 0.7)
        return chips, mult

    return BossDebuff(
        name='Heart Tax',
        description='Hands containing Hearts score 30% fewer chips.',
        apply_func=apply,
    )


def boss_half_chips():
    def apply(cards, hand_info, chips, mult, context):
        # all chip values halved
        chips = chips // 2
        return chips, mult

    return BossDebuff(
        name='Halved Winnings',
        description='All hands score half their normal chips.',
        apply_func=apply,
    )

In [151]:
def choose_boss_debuff(ante, rng):
    # could vary by ante later, but for now just random from a small pool
    options = [
        boss_no_discards,
        boss_debuff_hearts,
        boss_half_chips,
    ]
    factory = rng.choice(options)
    return factory()

In [152]:
def generate_shop(ante, rng):
    items: list[ShopItem] = []

    joker_factories = ALL_JOKER_FACTORIES[:]
    rng.shuffle(joker_factories)
    first_joker = joker_factories[0]()
    price = rng.randint(2, 4) + ante
    items.append(
        ShopItem(
            kind = 'joker',
            name = first_joker.name,
            price = price,
            payload = first_joker,
            description = first_joker.description,
        )
    )

    if rng.random() < 0.5 and len(ALL_VOUCHER_FACTORIES) > 0:
        # voucher
        v_factory = rng.choice(ALL_VOUCHER_FACTORIES)
        voucher = v_factory()
        price = rng.randint(4, 6) + ante  # a bit pricier
        items.append(
            ShopItem(
                kind = 'voucher',
                name = voucher.name,
                price = price,
                payload = voucher,
                description = voucher.description,
            )
        )
    else:
        # another Joker (if available)
        if len(ALL_JOKER_FACTORIES) > 1:
            second_factory = joker_factories[1]
        else:
            second_factory = joker_factories[0]
        joker2 = second_factory()
        price = rng.randint(4, 7) + ante
        items.append(
            ShopItem(
                kind = 'joker',
                name = joker2.name,
                price = price,
                payload = joker2,
                description = joker2.description,
            )
        )

    # --- one Planet ---
    hand_type = rng.choice(HAND_ORDER)
    upgrade_kind = rng.choice(['chips', 'mult'])
    planet_name = f'{hand_type.title()} Planet ({upgrade_kind})'
    planet_desc = (
        f'Upgrade {hand_type}: +10 chips'
        if upgrade_kind == 'chips'
        else f'Upgrade {hand_type}: +1 mult'
    )
    price = rng.randint(4, 7) + ante

    items.append(
        ShopItem(
            kind = 'planet',
            name = planet_name,
            price = price,
            payload = (hand_type, upgrade_kind),
            description = planet_desc,
        )
    )

    return items

In [153]:
class Strategy:
    name: str = 'Base'

    def choose_shop_item(self,
                         player: PlayerState,
                         items: list['ShopItem'],
                         jokers: list['Joker'],
                         hand_chips_upgrades: HandChipsUpgrades,
                         hand_mult_upgrades: HandMultUpgrades,
                         rng: random.Random) -> int | None:
        
        # prefer cheapest Joker, then cheapest Voucher, then cheapest Planet.
        def effective_price(it):
            return max(1, it.price - player.shop_discount)

        affordable = [ (idx, it) for idx, it in enumerate(items)
                       if effective_price(it) <= player.money ]
        if not affordable:
            return None

        jokers_aff = [ (idx, it) for idx, it in affordable if it.kind == 'joker' ]
        vouchers_aff = [ (idx, it) for idx, it in affordable if it.kind == 'voucher' ]
        planets_aff = [ (idx, it) for idx, it in affordable if it.kind == 'planet' ]

        if jokers_aff:
            idx, _ = min(jokers_aff, key = lambda pair: effective_price(pair[1]))
            return idx
        if vouchers_aff:
            idx, _ = min(vouchers_aff, key = lambda pair: effective_price(pair[1]))
            return idx
        if planets_aff:
            idx, _ = min(planets_aff, key = lambda pair: effective_price(pair[1]))
            return idx

        return None
    
    def should_discard(self,
                       player,
                       hand,
                       hand_info,
                       discards_left,
                       blind,
                       jokers,
                       run_context):
        return False


class PairStrategy(Strategy):
    name = 'Pair / Trips Focus'

    def choose_shop_item(self,
                         player,
                         items,
                         jokers,
                         hand_chips_upgrades,
                         hand_mult_upgrades,
                         rng: random.Random):
        def effective_price(it):
            return max(1, it.price - player.shop_discount)

        affordable = [
            (idx, it) for idx, it in enumerate(items)
            if effective_price(it) <= player.money
        ]
        if not affordable:
            return None

        pair_keywords = ('Jolly', 'Zany', 'pair', 'Three of a Kind', 'Full House')

        # prefer pair-related jokers
        pair_jokers = [
            (idx, it) for idx, it in affordable
            if it.kind == 'joker' and any(kw in it.name for kw in pair_keywords)
        ]
        if pair_jokers:
            idx, _ = min(pair_jokers, key=lambda pair: effective_price(pair[1]))
            return idx

        # then, prefer Planets that upgrade pair/trips-ish hands
        pairish_hand_types = {'pair', 'two_pair', 'three_of_a_kind', 'full_house', 'four_of_a_kind'}
        planets = [
            (idx, it) for idx, it in affordable
            if it.kind == 'planet' and any(ht in it.name.lower() for ht in pairish_hand_types)
        ]
        if planets:
            idx, _ = min(planets, key=lambda pair: effective_price(pair[1]))
            return idx

        # last, fall back to base behavior
        return super().choose_shop_item(player, items, jokers, hand_chips_upgrades, hand_mult_upgrades, rng)
    
    def should_discard(self,
                       player,
                       hand,
                       hand_info,
                       discards_left,
                       blind,
                       jokers,
                       run_context):
        if discards_left <= 0:
            return False
        # if it's not at least a pair, toss it and try again
        return hand_info['name'] not in PAIRISH_HANDS
    
class ScalingJokerStrategy(Strategy):
    name = "Scaling Joker Focus"

    def choose_shop_item(self, player, items, jokers,
                         hand_chips_upgrades, hand_mult_upgrades, rng):
        def price(it: ShopItem) -> int:
            return max(1, it.price - player.shop_discount)

        affordable = [(i, it) for i, it in enumerate(items) if price(it) <= player.money]
        if not affordable:
            return None

        # high priority scaling joker names
        priority_names = ("Green Joker", "Supernova", "Bull", "Bootstraps", "Stuntman Lite")

        prio_jokers = [
            (i, it) for i, it in affordable
            if it.kind == 'joker' and any(name in it.name for name in priority_names)
        ]
        if prio_jokers:
            i, _ = min(prio_jokers, key=lambda pair: price(pair[1]))
            return i

        # otherwise, fall back to base behavior (joker > voucher > planet)
        return super().choose_shop_item(player, items, jokers,
                                        hand_chips_upgrades, hand_mult_upgrades, rng)
    def should_discard(self,
                       player,
                       hand,
                       hand_info,
                       discards_left,
                       blind,
                       jokers,
                       run_context):
        if discards_left <= 0:
            return False
        # only discard really bad high-card hands
        if hand_info['name'] == 'high_card':
            total_rank = sum(RANK_POINTS[c.rank] for c in hand)
            return total_rank < 35  # arbitrary threshold
        return False

    
class FlushStrategy(Strategy):
    name = 'Flush / Suit Focus'

    def choose_shop_item(self, player, items, jokers,
                         hand_chips_upgrades, hand_mult_upgrades, rng):
        def price(it):
            return max(1, it.price - player.shop_discount)

        affordable = [(i, it) for i, it in enumerate(items) if price(it) <= player.money]
        if not affordable:
            return None

        flush_names = ('Smeared', 'Arrowhead', 'Wrathful')
        # prefer flush/suit jokers
        flush_jokers = [
            (i, it) for i, it in affordable
            if it.kind == 'joker' and any(n in it.name for n in flush_names)
        ]
        if flush_jokers:
            i, _ = min(flush_jokers, key=lambda pair: price(pair[1]))
            return i

        # prefer Planets for flush/straight_flush
        planets = [
            (i, it) for i, it in affordable
            if it.kind == 'planet' and ('flush' in it.name.lower())
        ]
        if planets:
            i, _ = min(planets, key=lambda pair: price(pair[1]))
            return i

        # fallback: base behavior
        return super().choose_shop_item(player, items, jokers,
                                        hand_chips_upgrades, hand_mult_upgrades, rng)
    
    def should_discard(self,
                       player,
                       hand,
                       hand_info,
                       discards_left,
                       blind,
                       jokers,
                       run_context):
        if discards_left <= 0:
            return False
        # if it's not a flush, keep trying
        return not hand_info['is_flush']
    
class StraightStrategy(Strategy):
    name = 'Straight / Shortcut Focus'

    def choose_shop_item(self, player, items, jokers,
                         hand_chips_upgrades, hand_mult_upgrades, rng):
        def price(it: ShopItem) -> int:
            return max(1, it.price - player.shop_discount)

        affordable = [(i, it) for i, it in enumerate(items) if price(it) <= player.money]
        if not affordable:
            return None

        straight_names = ('Shortcut', 'Runner')

        # prefer straight jokers
        straight_jokers = [
            (i, it) for i, it in affordable
            if it.kind == 'joker' and any(n in it.name for n in straight_names)
        ]
        if straight_jokers:
            i, _ = min(straight_jokers, key=lambda pair: price(pair[1]))
            return i

        # then, planets for straight / straight_flush
        planets = [
            (i, it) for i, it in affordable
            if it.kind == 'planet' and any(ht in it.name.lower()
                                           for ht in ('straight', 'straight_flush'))
        ]
        if planets:
            i, _ = min(planets, key=lambda pair: price(pair[1]))
            return i

        return super().choose_shop_item(player, items, jokers,
                                        hand_chips_upgrades, hand_mult_upgrades, rng)
    
    def should_discard(self,
                       player,
                       hand,
                       hand_info,
                       discards_left,
                       blind,
                       jokers,
                       run_context):
        if discards_left <= 0:
            return False
        values = [RANK_POINTS[c.rank] for c in hand]
        if hand_info['is_straight'] or _is_almost_straight(values):
            return False
        return True

class FaceStrategy(Strategy):
    name = 'Pareidolia / Face Focus'

    def choose_shop_item(self, player, items, jokers,
                         hand_chips_upgrades, hand_mult_upgrades, rng):
        def price(it: ShopItem) -> int:
            return max(1, it.price - player.shop_discount)

        affordable = [(i, it) for i, it in enumerate(items) if price(it) <= player.money]
        if not affordable:
            return None

        face_names = ('Pareidolia', 'Scary Face', 'Smiley Face')

        # prefer Pareidolia first
        pareidolia = [
            (i, it) for i, it in affordable
            if it.kind == 'joker' and 'Pareidolia' in it.name
        ]
        if pareidolia:
            i, _ = min(pareidolia, key=lambda pair: price(pair[1]))
            return i

        # then, face jokers
        face_jokers = [
            (i, it) for i, it in affordable
            if it.kind == 'joker' and any(n in it.name for n in face_names)
        ]
        if face_jokers:
            i, _ = min(face_jokers, key=lambda pair: price(pair[1]))
            return i

        return super().choose_shop_item(player, items, jokers,
                                        hand_chips_upgrades, hand_mult_upgrades, rng)
    
    def should_discard(self,
                       player,
                       hand,
                       hand_info,
                       discards_left,
                       blind,
                       jokers,
                       run_context):
        if discards_left <= 0:
            return False
        # if we have Pareidolia active, never discard
        run = run_context
        if run.get('pareidolia_active'):
            return False

        face_count = sum(1 for c in hand if c.rank in FACE_RANKS)
        # try again if we don't have faces
        return face_count == 0



In [154]:
def run_shop_phase(player,
                   jokers,
                   hand_chips_upgrades,
                   hand_mult_upgrades,
                   ante,
                   rng,
                   strategy,
                   verbose: bool = True):
    items = generate_shop(ante, rng)

    if verbose:
        print('\n=== Shop Phase ===')
        print(f'Money: {player.money}')
        for i, it in enumerate(items):
            effective_price = max(1, it.price - player.shop_discount)
            print(f'[{i}] {it.name} ({it.kind}) - {effective_price}$  :: {it.description}')

    # ask strategy which to buy
    choice_idx = strategy.choose_shop_item(
        player,
        items,
        jokers,
        hand_chips_upgrades,
        hand_mult_upgrades,
        rng,
    )

    if choice_idx is None:
        if verbose:
            print('Strategy chose to buy nothing. Leaving shop.')
        return player, jokers, hand_chips_upgrades, hand_mult_upgrades

    if not (0 <= choice_idx < len(items)):
        if verbose:
            print(f'Strategy returned invalid index {choice_idx}. Leaving shop.')
        return player, jokers, hand_chips_upgrades, hand_mult_upgrades

    choice = items[choice_idx]
    effective_price = max(1, choice.price - player.shop_discount)

    if effective_price > player.money:
        if verbose:
            print(f'Strategy tried to buy {choice.name} but cannot afford it.')
        return player, jokers, hand_chips_upgrades, hand_mult_upgrades

    # enforce Joker slot limit
    if choice.kind == 'joker' and len(jokers) >= player.max_jokers:
        if verbose:
            print(f'Cannot buy {choice.name}: Joker slots full ({player.max_jokers}).')
        return player, jokers, hand_chips_upgrades, hand_mult_upgrades

    # buy
    player.money -= effective_price

    if choice.kind == 'joker':
        jokers.append(choice.payload)
        if verbose:
            print(f'Bought Joker: {choice.name} for {effective_price}$')
    elif choice.kind == 'planet':
        hand_type, upgrade_kind = choice.payload
        apply_planet_upgrade(hand_type, upgrade_kind,
                             hand_chips_upgrades, hand_mult_upgrades)
        if verbose:
            print(f'Bought Planet: {choice.name} for {effective_price}$ -> '
                  f'upgraded {hand_type} ({upgrade_kind})')
    elif choice.kind == 'voucher':
        voucher: Voucher = choice.payload
        voucher.apply(player)
        if verbose:
            print(f'Bought Voucher: {voucher.name} for {effective_price}$ -> '
                  f'{voucher.description}')

    return player, jokers, hand_chips_upgrades, hand_mult_upgrades

In [155]:
def play_blind(player,
               blind,
               jokers,
               hand_chips_upgrades,
               hand_mult_upgrades,
               rng,
               run_context,
               strategy,
               verbose: bool = True):
    if verbose:
        print(f'\n=== Ante {blind.ante} - {blind.name} Blind ===')
        print(f'Target chips: {blind.target_chips}')
        print(f'Hands: {blind.hands}, Discards: {blind.discards}')

    deck = new_deck(shuffle=True, seed=rng.randint(0, 10**9))

    chips_this_blind = 0
    hands_left = blind.hands
    discards_left = blind.discards

    played_hands = 0

    while hands_left > 0:
        if len(deck) < 5:
            deck = new_deck(shuffle=True, seed=rng.randint(0, 10**9))

        hand = draw_cards(deck, 5)
        info = evaluate_hand(hand)

        # discard decision
        if discards_left > 0 and strategy.should_discard(
            player,
            hand,
            info,
            discards_left,
            blind,
            jokers,
            run_context,
        ):
            if verbose:
                card_str = ', '.join(str(c) for c in hand)
                print(f'\nDiscarding hand (discards left {discards_left - 1}): '
                      f'{card_str} [{info["name"]}]')
            discards_left -= 1
            # try again without consuming a hand
            continue

        # play the hand
        run_context['global_hand_index'] += 1
        run_context['hands_played'] += 1
        hand_type = info['name']
        if 'hand_type_counts' in run_context:
            run_context['hand_type_counts'][hand_type] = (
                run_context['hand_type_counts'].get(hand_type, 0) + 1
            )
        run_context['player_money'] = player.money

        context = {
            'ante': blind.ante,
            'blind_name': blind.name,
            'hand_index_in_blind': played_hands,
            'run': run_context,
        }

        chips, mult, total = score_with_modifiers(
            hand, info, jokers, hand_chips_upgrades, hand_mult_upgrades, context
        )

        chips_this_blind += total
        hands_left -= 1
        played_hands += 1

        if verbose:
            card_str = ', '.join(str(c) for c in hand)
            print(f'\nHand played ({played_hands}/{blind.hands}): {card_str}')
            print(f'Type: {info["name"]}')
            print(f'Hand score: {total}  (chips = {chips}, mult = {mult})')
            print(f'Accumulated chips: {chips_this_blind}/{blind.target_chips}')

        if chips_this_blind >= blind.target_chips:
            if verbose:
                print(f'\n>>> Cleared {blind.name} Blind! <<<')

            if blind.name == 'Small':
                reward = 5 + 2 * blind.ante
            elif blind.name == 'Big':
                reward = 7 + 3 * blind.ante
            else:
                reward = 10 + 4 * blind.ante

            player.money += reward
            return player, True

    # we ran out of hands and failed the blind
    player.hp -= 1
    if verbose:
        print(f'\nXXX Failed {blind.name} Blind. Lost 1 HP. Remaining HP: {player.hp}')
    return player, False


In [156]:
def play_run(strategy,
             max_ante: int = 3,
             seed: int | None = 12345,
             verbose: bool = True) -> RunResult:
    player = PlayerState(hp = 3, money = 0, ante = 1, blind_index = 0)

    jokers: list[Joker] = []
    hand_chips_upgrades: HandChipsUpgrades = {}
    hand_mult_upgrades: HandMultUpgrades = {}

    rng = random.Random(seed)
    run_context = new_run_context()

    blinds_cleared = 0
    max_ante_reached = player.ante

    while player.hp > 0 and player.ante <= max_ante:
        blinds = make_blinds_for_ante(player.ante)

        for idx, blind in enumerate(blinds):
            player.blind_index = idx

            player, success = play_blind(
                player,
                blind,
                jokers,
                hand_chips_upgrades,
                hand_mult_upgrades,
                rng,
                run_context,
                strategy,
                verbose=verbose,
                )


            if not success:
                # apply interest once when you fail a blind
                apply_interest(player)
                if verbose:
                    if player.hp <= 0:
                        print('\n=== Run Over: Out of HP ===')
                    else:
                        print('\n=== Run Over: Failed a blind ===')

                return RunResult(
                    player = player,
                    jokers = jokers,
                    hand_chips_upgrades = hand_chips_upgrades,
                    hand_mult_upgrades = hand_mult_upgrades,
                    blinds_cleared = blinds_cleared,
                    max_ante_reached = max_ante_reached,
                )

            # shop after small and big blinds
            if blind.name in {'Small', 'Big'}:
                player, jokers, hand_chips_upgrades, hand_mult_upgrades = run_shop_phase(
                    player,
                    jokers,
                    hand_chips_upgrades,
                    hand_mult_upgrades,
                    player.ante,
                    rng,
                    strategy,
                    verbose = verbose,
                )

            # apply interest at end of blind
            apply_interest(player)

            blinds_cleared += 1

        if verbose:
            print(f'\n=== Cleared Ante {player.ante}! Moving to Ante {player.ante + 1} ===')
            print('Current Jokers:')
            for j in jokers:
                print(' -', j.name)
            print('Hand upgrades (chips):', hand_chips_upgrades)
            print('Hand upgrades (mult):', hand_mult_upgrades)

        # move to next ante
        player.ante += 1
        player.blind_index = 0
        max_ante_reached = max(max_ante_reached, player.ante)

    if verbose:
        if player.ante > max_ante:
            print('\n=== Run Complete: Reached max ante! ===')
        else:
            print('\n=== Run Ended Early ===')

    # final wrap in RunResult
    return RunResult(
        player = player,
        jokers = jokers,
        hand_chips_upgrades = hand_chips_upgrades,
        hand_mult_upgrades = hand_mult_upgrades,
        blinds_cleared = blinds_cleared,
        max_ante_reached = max_ante_reached,
    )

In [157]:
def simulate_runs(strategy,
                  n_runs: int = 1000,
                  max_ante: int = 3,
                  base_seed: int = 1000,
                  verbose: bool = False):
    
    results: list[RunResult] = []

    for i in range(n_runs):
        seed = base_seed + i
        res = play_run(strategy, max_ante = max_ante, seed = seed, verbose = verbose)
        results.append(res)

    # win is if you beat all blinds up to it
    wins = sum(1 for r in results if r.max_ante_reached > max_ante)
    avg_final_ante = sum(r.max_ante_reached for r in results) / n_runs
    avg_blinds = sum(r.blinds_cleared for r in results) / n_runs
    avg_money = sum(r.player.money for r in results) / n_runs

    summary = {
        'strategy': strategy.name,
        'n_runs': n_runs,
        'win_rate': wins / n_runs,
        'avg_final_ante': avg_final_ante,
        'avg_blinds_cleared': avg_blinds,
        'avg_money': avg_money,
    }
    return results, summary


In [158]:
if __name__ == '__main__':
    print('== Demo hands ==')
    test_draw_and_score(seed=42)
    test_draw_and_score(seed=6644)

    strategies = [
        PairStrategy(),
        ScalingJokerStrategy(),
        FlushStrategy(),
        StraightStrategy(),
        FaceStrategy(),
    ]

    print('\n== Monte Carlo: 5 archetypes (n=1000) ==')
    for strat in strategies:
        _, summary = simulate_runs(
            strat,
            n_runs=1000,
            max_ante=3,
            base_seed=1000 + hash(strat.name) % 1000,
            verbose=False,
        )
        print(summary)

== Demo hands ==
Hand: 6 of Hearts, K of Clubs, 2 of Hearts, 3 of Clubs, Q of Spades
Type: high_card
Chips: 46 | Mult: 1 | Score: 46
Hand: 9 of Hearts, 9 of Spades, K of Spades, J of Hearts, 8 of Spades
Type: pair
Chips: 70 | Mult: 1 | Score: 70

== Monte Carlo: 5 archetypes (n=1000) ==
{'strategy': 'Pair / Trips Focus', 'n_runs': 1000, 'win_rate': 0.26, 'avg_final_ante': 1.859, 'avg_blinds_cleared': 3.44, 'avg_money': 37.818}
{'strategy': 'Scaling Joker Focus', 'n_runs': 1000, 'win_rate': 0.435, 'avg_final_ante': 2.357, 'avg_blinds_cleared': 4.686, 'avg_money': 66.234}
{'strategy': 'Flush / Suit Focus', 'n_runs': 1000, 'win_rate': 0.236, 'avg_final_ante': 1.786, 'avg_blinds_cleared': 3.142, 'avg_money': 37.974}
{'strategy': 'Straight / Shortcut Focus', 'n_runs': 1000, 'win_rate': 0.275, 'avg_final_ante': 1.905, 'avg_blinds_cleared': 3.461, 'avg_money': 43.057}
{'strategy': 'Pareidolia / Face Focus', 'n_runs': 1000, 'win_rate': 0.361, 'avg_final_ante': 2.176, 'avg_blinds_cleared': 4.22